In [5]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import pandas as pd
import requests
from newspaper import Article
import numpy as np

In [6]:
def sentimentAnalysis(text, dictionary, url):
    if(len(dictionary) == 0):
        dictionary = {
            "URL": [],
            "Sentiment Score": [],
            "Sentiment Label": [],
            "Subjectivity Score": [],
            "Positive Words": [],
            "Negative Words": []
            }
    if(text[0:8] != "PARERROR"):
        # Start the sentiment analysis now
        dictionary["URL"].append(url)
        doc = nlp(text)
        sentiment = doc._.blob.polarity
        sentiment = round(sentiment,2)
        subjectivity = doc._.blob.subjectivity
        subjectivity = round(subjectivity,2)

        # Gives positive or negative label
        if sentiment >= 0.033 and sentiment <= 0.043:
            sent_label = "Neutral"
        elif sentiment > 0.043 and sentiment < 0.143:
            sent_label = "Neutral Positive"
        elif sentiment > 0.143:
            sent_label = "Positive"
        elif sentiment < 0.033 and sentiment > -0.062:
            sent_label = "Neutral Negative"
        elif sentiment < -0.062:
            sent_label = "Negative"

        dictionary["Sentiment Label"].append(sent_label)
        dictionary["Sentiment Score"].append(sentiment)
        dictionary["Subjectivity Score"].append(subjectivity)

        positive_words = []
        negative_words = []

        for x in doc._.blob.sentiment_assessments.assessments:
          if x[1] > 0:
            positive_words.append(x[0][0])
          elif x[1] < 0:
            negative_words.append(x[0][0])
          else:
            pass

        dictionary["Positive Words"].append(', '.join(set(positive_words)))
        dictionary["Negative Words"].append(', '.join(set(negative_words)))
    
    else:
        dictionary["URL"].append(url)
        dictionary["Sentiment Label"].append(text)
        dictionary["Sentiment Score"].append(0.0)
        dictionary["Subjectivity Score"].append(0.0)

        positive_words = []
        negative_words = []

        dictionary["Positive Words"].append(', '.join(set(positive_words)))
        dictionary["Negative Words"].append(', '.join(set(negative_words)))
    
    return dictionary

In [7]:
def topic_level_sentiment(ldamodel):
    my_dict = {'Topic_' + str(i): [token for token, score in ldamodel.show_topic(i, topn=10)] for i in range(0, ldamodel.num_topics)}
    
    return my_dict

In [8]:
def get_sentences(doc):
    return doc.sents

#Takes a doc object from spacy and returns a tuple list of form (sentence, sentiment of sentence) for all sentences
def sentence_sentiment_from_doc(doc):
    sentences = get_sentences(doc)
    tuple_list = []
    for sentence in sentences:
        sent_doc = nlp(sentence.text)
        tuple_list.append((sentence.text,sent_doc._.blob.polarity))
    return tuple_list


# Finds all named entities (does not filter by person, date, etc) and returns sentiment value based on the sentences they
# are included in. Returns dictionary of form {entity name : sentiment value}
def sentence_sentiment_on_named_entities(lda):
    doc = nlp(lda)
    named_ent_list = topic_level_sentiment(doc)
    sentence_sentiment_list = sentence_sentiment_from_doc(doc)
    return_dict = {}
    
    for (name, label) in named_ent_list:
        return_dict[name] = 0
        
        for (sentence, sentiment) in sentence_sentiment_list:
             if sentence.find(name) != -1:
                    return_dict[name] += sentiment